Source: https://github.com/UKPLab/sentence-transformers

In [1]:
!pip install pycarol
!pip install -U sentence-transformers

Requirement already up-to-date: sentence-transformers in /Users/felipe/opt/miniconda3/envs/recommendation/lib/python3.8/site-packages (0.2.6.1)


In [4]:
from pycarol import Carol, Staging, ApiKeyAuth, Storage, PwdAuth
import pandas as pd
import pickle
import numpy as np
import re
import math

In [5]:
login_dont = Carol(domain='monitoriaqa',
app_name=' ',
organization='totvs',
auth=PwdAuth('fmendes@totvs.com.br', 'Intxdx70248821*'))

In [67]:
df = Staging(login_dont).fetch_parquet(staging_name='articles',
                        connector_name='carol_connect_zerado',
                        cds=True
                        )

100%|██████████| 71/71 [01:31<00:00,  1.28s/it]


In [68]:
df = df.astype(str)

In [69]:
df.shape

(36332, 26)

In [70]:
df = df.replace('', np.nan)

In [71]:
df = df[(df.source_locale == 'pt-br') & (~df.body.isnull())]

In [72]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    text = re.sub(clean, '', text)
    return " ".join(re.sub(r'\s([?.!"](?:\s|$))', r'\1', text).strip().split())

In [73]:
def get_question(body):
    
    body = body.replace('\n', '').replace('<br>', '')

    m = re.search('(?<=<strong>D(ú|u)vida).*?(?=<strong>Ambiente)', body, re.IGNORECASE)
    if m:
        return remove_html_tags(m.group(0))
        
    m = re.search('(?<=<strong>Ocorr(ê|e)ncia).*?(?=<strong>Ambiente)', body, re.IGNORECASE)
    if m:
        return remove_html_tags(m.group(0))
    
    return np.nan

In [74]:
def get_question_type(body):
    
    body = body.replace('\n', '').replace('<br>', '')

    m = re.search('(?<=<strong>D(ú|u)vida).*?(?=<strong>Ambiente)', body, re.IGNORECASE)
    if m:
        return 'question'

    m = re.search('(?<=<strong>Ocorr(ê|e)ncia).*?(?=<strong>Ambiente)', body, re.IGNORECASE)
    if m:
        return 'occurrence'
    
    return np.nan

In [75]:
def get_environment(body):
    
    body = body.replace('\n', '').replace('<br>', '')

    m = re.search('(?=<strong>Ambiente).*?(?=<strong>Solu(ç|c)(ã|a)o)', body, re.IGNORECASE)
    if not m:
        return np.nan
    return remove_html_tags(m.group(0))

In [76]:
def get_solution(body):
    
    body = body.replace('\n', '').replace('<br>', '')

    m = re.search('(?<=<strong>Solu(ç|c)(ã|a)o)(?s)(.*$)', body, re.IGNORECASE)
    if not m:
        return np.nan
    return m.group(0)

In [77]:
def get_sanitized_solution(body):
    
    body = body.replace('\n', '').replace('<br>', '')

    m = re.search('(?<=<strong>Solu(ç|c)(ã|a)o)(?s)(.*$)', body, re.IGNORECASE)
    if not m:
        return np.nan
    return remove_html_tags(m.group(0))

In [78]:
df['question'] = df.body.apply(get_question)
df['question_type'] = df.body.apply(get_question_type)
df['environment'] = df.body.apply(get_environment)
df['solution'] = df.body.apply(get_solution)
df['sanitized_solution'] = df.body.apply(get_sanitized_solution)

In [79]:
df = df.replace('', np.nan)

In [80]:
df.head()

,__v,author_id,body,brand_id,brand_name,comments_disabled,created_at,draft,edited_at,html_url,...,updated,updated_at,url,vote_count,vote_sum,question,question_type,environment,solution,sanitized_solution
0,0,19660061288.0,<p><br><strong>Dúvida</strong><strong><br></st...,2911068,Fluig,False,2019-08-27T16:20:33.000Z,False,2019-08-27T16:20:37.000Z,https://centraldeatendimento.fluig.com/hc/pt-b...,...,False,2019-08-27T16:20:37.000Z,https://fluigexterno.zendesk.com/api/v2/help_c...,0,0,Como faço para configurar o widget Analytics e...,question,Ambientefluig - Analytics - A partir da versão...,</strong><strong></strong>Siga os passos abaix...,Siga os passos abaixo para a configuração do w...
1,0,19660061288.0,<p><br><strong>Dúvida</strong><strong><br></st...,2911068,Fluig,False,2019-08-27T14:35:14.000Z,False,2019-08-27T14:36:32.000Z,https://centraldeatendimento.fluig.com/hc/pt-b...,...,False,2019-08-27T14:36:32.000Z,https://fluigexterno.zendesk.com/api/v2/help_c...,0,0,Ao tentar alterar as informações de um usuário...,question,Ambientefluig - Configuração de uso - A partir...,</strong>Detectamos que essa situação também p...,Detectamos que essa situação também pode ocorr...
2,0,19660061288.0,<p><strong><br>Ocorrência</strong><strong><br>...,2911068,Fluig,False,2019-08-22T17:27:14.000Z,False,2020-03-19T13:50:44.000Z,https://centraldeatendimento.fluig.com/hc/pt-b...,...,False,2020-03-19T13:50:44.000Z,https://fluigexterno.zendesk.com/api/v2/help_c...,0,0,"Usei o recurso ""Esqueceu Sua Senha"", mas receb...",occurrence,Ambientefluig - Identity – Todas as versões Ca...,</strong><strong></strong></p><p>O link de red...,O link de redefinição de senha enviado para o ...
3,0,19660061288.0,<p><br><strong>Dúvida</strong><strong><br></st...,2911068,Fluig,False,2019-10-11T20:04:52.000Z,False,2019-10-11T20:04:56.000Z,https://centraldeatendimento.fluig.com/hc/pt-b...,...,False,2019-10-11T20:20:28.000Z,https://fluigexterno.zendesk.com/api/v2/help_c...,1,1,A mensagem FLUIG_REST_CLIENT_500 é mostrada no...,question,Ambientefluig - Arquitetura e ambiente - Todas...,</strong><strong></strong>A inicialização dos ...,"A inicialização dos serviços é assíncrona, de ..."
4,0,365733291352.0,<p><strong>Dúvida<br></strong>Um dos nossos us...,2911068,Fluig,False,2019-08-22T16:57:02.000Z,False,2019-09-13T20:08:18.000Z,https://centraldeatendimento.fluig.com/hc/pt-b...,...,False,2019-09-13T20:08:18.000Z,https://fluigexterno.zendesk.com/api/v2/help_c...,0,0,Um dos nossos usuários está cadastrado no flui...,question,Ambientefluig - Identity - A partir da versão ...,</strong>O procedimento será diferente se o id...,O procedimento será diferente se o identity es...


In [81]:
df = df.dropna(subset=['question', 'question_type', 'environment', 'solution'])

In [82]:
df = df.reset_index(drop=True)

In [83]:
df['question'].count()

32540

In [84]:
df['id'] = df['id'].apply(lambda x: x.replace('.0', ''))

In [85]:
df.iloc[0].id

'360034984293'

In [86]:
dupl = df[df.duplicated(['question'])]

In [87]:
dupl[['question', 'brand_id', 'label_names', 'title', 'html_url']].to_excel('articles_with_duplicate_questions.xlsx', index=False)

In [88]:
dupl.head()[['question', 'brand_id', 'label_names', 'title']]

,question,brand_id,label_names,title
231,Quais as recomendações devem ser seguidas para...,2911068,"[""processo"",""solicitação"",""faqfluig"",""bpm"",""as...",FL - BPM - Dividir múltiplas integrações de um...
233,Quais as recomendações devem ser seguidas para...,2911068,"[""serviço"",""solicitação"",""faqfluig"",""bpm"",""eve...",FL - BPM - Tarefa de serviço assíncrono para c...
237,Quais as recomendações devem ser seguidas para...,2911068,"[""integração"",""dataset"",""converter"",""processos...",FL - BPM - Transferir código de integração de ...
464,É possível retirar a permissão do usuário para...,1509248,"[""inserir_label_produto"",""inserir_label_rotina...",GA - TOTVS DF-e - Permissão para visualizar ou...
770,[Exemplo:Qual a pausa telefônica utilizada qua...,1509248,"[""inserir_label_produto"",""inserir_label_rotina...",ABREV. PRODUTO - ABREV. MÓDULO - Descrição Dúv...


In [89]:
dupl.iloc[1].html_url

'https://centraldeatendimento.fluig.com/hc/pt-br/articles/360027558711-FL-BPM-Tarefa-de-servi%C3%A7o-ass%C3%ADncrono-para-chamar-as-integra%C3%A7%C3%B5es-no-lugar-dos-eventos'

In [90]:
hifen = df[df.question.str.contains('((?:\w+-)+\w+)', regex=True)]

<ipython-input-90-0571c2c7d7f7>:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  hifen = df[df.question.str.contains('((?:\w+-)+\w+)', regex=True)]


In [91]:
hifen.shape

(3173, 31)

In [92]:
hifen.sample(5)[['question']]

,question
9567,Rejeição 1017 - NF-e Emitido em ambiente de pr...
6189,S-1200 - Como informar 13º Salário para Pró-la...
12263,MDTA640 - THREAD ERROR invalid class MDT640EVE...
4174,Quais os fontes envolvidos para cancelamento d...
4375,Como consultar no site do eSocial dados do eve...


In [93]:
hifen.iloc[3].question

'É possível alterar o endereço de e-mail de um usuário no Identity?'

In [94]:
hifen[['question', 'brand_id', 'label_names', 'title', 'html_url']].to_excel('articles_with_hyphenated_words_in_questions.xlsx', index=False)

In [95]:
target = Carol(domain='mendes',
app_name='searchskill',
organization='datascience',
auth=ApiKeyAuth('bf56760357424b60ae9558bcf9f090c2'),
connector_id='175e781f1cce470eb0457323430f1407')

In [96]:
target_stag = Staging(target)

In [97]:
target_stag.send_data(staging_name='articles', connector_id='175e781f1cce470eb0457323430f1407', data=df,
                             crosswalk_auto_create=['idinternalmdb'], auto_create_schema=True)

<ipython-input-97-33f5a54b4626>:1: DeprecationWarning: fields_dict will be deprecated, use `data`
  target_stag.send_data(staging_name='articles', connector_id='175e781f1cce470eb0457323430f1407', data=df,


provided crosswalk  ['idinternalmdb']


### Get sentence embeddings

In [98]:
from sentence_transformers import SentenceTransformer

In [99]:
# Load Sentence model (based on BERT) from URL
model = SentenceTransformer('distiluse-base-multilingual-cased')

In [101]:
# Embed a list of sentences
sentence_embeddings = model.encode(df.question.values)

In [102]:
len(sentence_embeddings)

32540

In [103]:
id_list = df['id'].values

In [104]:
len(id_list)

32540

In [105]:
index_to_question_id_mapping = dict(enumerate(id_list))
question_id_to_index_mapping = {j: i for i,j in index_to_question_id_mapping.items()} 

### Save objects in Storage

In [106]:
stg = Storage(target)

In [107]:
def save_object_to_storage(obj, filename):
    with open(filename, "bw") as f:
        pickle.dump(obj, f)
    stg.save(filename, obj, format='pickle')

In [109]:
save_object_to_storage(sentence_embeddings, 'sentence_embeddings')

In [110]:
save_object_to_storage(index_to_question_id_mapping, 'index_to_question_id_mapping')

In [111]:
save_object_to_storage(question_id_to_index_mapping, 'question_id_to_index_mapping')

In [20]:
from sentence_transformers import SentenceTransformer, LoggingHandler

In [21]:
# Load Sentence model (based on BERT) from URL
model = SentenceTransformer('distiluse-base-multilingual-cased')

In [22]:
sentence_embeddings = pickle.load(open("sentence_embeddings", "rb"))

In [ ]:
df.index

In [28]:
df.iloc[12].question

"Após configurar HTTPS no fluig, ao acessá-lo pelo aplicativo fluig Mobile e informar servidor de conexão é apresentado o erro 'Falha na comunicação'."

In [144]:
def get_most_similar_question(query):

    query_vec = model.encode([query])
    k = 5

    # compute normalized dot product as score
    score = np.sum(query_vec[0] * sentence_embeddings, axis=1) / np.linalg.norm(sentence_embeddings, axis=1)
    topk_scores = np.sort(score)[::-1]
    topk_idx = np.argsort(score)[::-1]
    return topk_idx[:k], topk_scores[:k]

In [119]:
#size = np.int32(np.floor(0.2 * questions.shape[0]))
size = 100

indices = np.random.choice(
    df.question.index, 
    replace=False,
    size=size
).tolist()

In [71]:
questions = []

In [93]:
df.iloc[indices[9]].question

'Solicito uma função para conversão de um arquivo .doc para .PDF'

In [94]:
questions.append('função para converter arquivo doc para pdf')

In [97]:
len(questions)

10

In [145]:
get_most_similar_question('adicionar transportadora nas notas fiscais geradas na rotina 1322')

(array([ 4270, 23653, 14174, 26242, 27618]),
 array([0.64584833, 0.60263   , 0.5729962 , 0.54728776, 0.54643303],
       dtype=float32))

In [150]:
df.iloc[4270].question

'Como inserir transportadora em notas fiscais geradas pela rotina 1322 - Simples Remessa Baixando Estoque.'

In [126]:
for i, query in enumerate(questions):
    raw_query = df.iloc[indices[i]].question
    most_similar = get_most_similar_question(query)
    if most_similar != i:
        print(f'{query}')
        print(f'{indices[i]} - {raw_query}')
        print(f'{most_similar} - {df.iloc[most_similar].question}')

0.645848274230957 - 0.645848274230957
0.6026300191879272 - 0.6026300191879272
0.5729961395263672 - 0.5729961395263672
0.5472877025604248 - 0.5472877025604248
0.5464330315589905 - 0.5464330315589905
0.5448733568191528 - 0.5448733568191528
0.5358591079711914 - 0.5358591079711914
0.531542956829071 - 0.531542956829071
0.5271217823028564 - 0.5271217823028564
0.5169530510902405 - 0.5169530510902405
adicionar transportadora nas notas fiscais geradas na rotina 1322
17835 - Por que um determinado lançamento não é apresentado na rotina 723?
4270 - Como inserir transportadora em notas fiscais geradas pela rotina 1322 - Simples Remessa Baixando Estoque.
0.5572085976600647 - 0.5572085976600647
0.5351568460464478 - 0.5351568460464478
0.5280529856681824 - 0.5280529856681824
0.5269234776496887 - 0.5269234776496887
0.5140586495399475 - 0.5140586495399475
0.5049834847450256 - 0.5049834847450256
0.49780139327049255 - 0.49780139327049255
0.4954758882522583 - 0.4954758882522583
0.49390849471092224 - 0.4939

In [115]:
df.iloc[23587].html_url

'https://centraldeatendimento.totvs.com/hc/pt-br/articles/360035902474-LG-PAT-Formas-de-transfer%C3%AAncia-de-um-bem-para-outro-na-mesma-empresa'

In [114]:
df.iloc[17132].html_url

'https://centraldeatendimento.totvs.com/hc/pt-br/articles/360025943751-LG-PAT-Como-efetuar-a-transfer%C3%AAncia-de-bens-entre-empresas'

In [112]:
df.iloc[1527].html_url

'https://centraldeatendimento.totvs.com/hc/pt-br/articles/360045420414-WINT-Como-utilizar-a-rotina-2039-Cadastro-de-Finalizadora'

In [125]:
from pycarol import Query

In [113]:
from pycarol.filter import Filter, TYPE_FILTER, TERMS_FILTER

In [151]:
def get_similar_questions(model, sentence_embeddings, query, k):
    query_vec = model.encode([query])
    score = np.sum(query_vec[0] * sentence_embeddings, axis=1) / np.linalg.norm(sentence_embeddings, axis=1)
    topk_scores = np.sort(score)[::-1]
    topk_idx = np.argsort(score)[::-1]
    print(topk_idx[:k], topk_scores[:k])
    return list(topk_idx[:k]), list(topk_scores[:k])

In [131]:
def get_questions_by_ids(question_ids):
    dm_name = 'articles'
    query_filter = Filter.Builder().must(TYPE_FILTER(value=f'{dm_name}Golden')) \
        .must(TERMS_FILTER(key='mdmGoldenFieldAndValues.id.raw', value=question_ids)).build().to_json()
    page_size=1000
    return Query(target, page_size=page_size).query(query_filter).go().results

In [165]:
def query(query):
    k = 5
    topk_idx, topk_scores = get_similar_questions(model, sentence_embeddings, query, k)
    question_ids = [index_to_question_id_mapping[idx] for idx in topk_idx]
    results = get_questions_by_ids(question_ids)
    response = ''
    for i, question_id in enumerate(question_ids):
        result = [result for result in results if result['id'] == question_id][0]
        response = response + 'Pergunta similar: ' + result['question'] + '\n'
        response = response + 'Resposta: ' + result['solution'] + '\n'
        response = response + 'Score: ' + str(topk_scores[topk_idx.index(question_id_to_index_mapping[question_id])])
        if i != 4:
            response = response +  '\n\n'
    # for i, idx in enumerate(topk_idx):
    #     question_id = index_to_question_id_mapping[idx]
    #     question = get_question_by_id(question_id)[0]
    #     response = response + 'Pergunta similar: ' + question.question + '\n'
    #     response = response + 'Resposta: ' + question.solution + '\n'
    #     response = response + 'Score: ' + topk_scores[i]
    #     if i != 4:
    #         response = response +  '\n\n'
    return {'response': response}

In [166]:
query('adicionar transportadora nas notas fiscais geradas na rotina 1322')

[ 4270 23653 14174 26242 27618] [0.64584833 0.60263    0.5729962  0.54728776 0.54643303]


{'response': 'Pergunta similar: Como inserir transportadora em notas fiscais geradas pela rotina 1322 - Simples Remessa Baixando Estoque.\nResposta: </strong><strong></strong>A rotina <strong>1322</strong> grava a <strong>Transportadora Padrão</strong> nas notas fiscais quando o parâmetro\xa0<strong>2159-Utilizar transportador padrão</strong> da rotina <strong>132-Parâmetros da Presidência</strong> (configuração por filial) está marcado como\xa0<strong>Sim</strong>, lembrando que os dados da transportadora devem ser cadastrados na rotina<strong> 535-Cadastro de Filiais</strong> aba <strong>Dados da Transportadora</strong>.</p><p>Essas informações serão destacadas na rotina<strong> 1452-Emitir documento eletrônico</strong>, no campo <strong>Transportador/Volumes Transportados</strong>.</p>\nScore: 0.64584833\n\nPergunta similar: Ao gerar uma transferência na rotina 1436 - Gerar nota fiscal de transferência para depósito\nResposta: </strong><strong></strong><span>Para resolver essa mensa

In [167]:
import requests

In [170]:
query = 'adicionar transportadora nas notas fiscais geradas na rotina 1322'

In [174]:
data = {
    'query': query
}

response = requests.post(url='http://localhost:5000/query',
                                 json=data)
print(response)
print(response.text)
response.json()

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))